# Goal: simulate market dataset such that we can correctly estimate coefficients later on 



In [17]:
import import_ipynb
import BLP_functions as f 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from numpy.linalg import inv
import builtins

np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(legacy='1.13')

n_market = 50
n_product = 5
n_consumer = 100
market_variance = 0.5 #amount of variation between markets 
endogenous_noise = 0.5 #how noise to generate price 

#structural parameters 
true_alpha = 3 
true_beta = 7 
true_sigma_alpha = 0.7 
true_sigma_beta = 0.3
market_variance = 0.3 #amount of variation between markets 

In [18]:
all_markets = []
for market in range(n_market):
    x = np.random.uniform(0, 1, n_product)
    instrument = np.random.uniform(0, 10, n_product)

    # generate market shares and price variables endogenously 
    price = f.price_gen(x, instrument, endogenous_noise)
    inner_utility = f.utility_gen([true_alpha, true_beta], [true_sigma_alpha, true_sigma_beta], x, price)
    inner_market_utility = f.market_gen(inner_utility, market_variance)
    inner_prob_choice = f.choice_prob(pd.DataFrame(inner_market_utility))
    # generate market share directly from the choice probabilities
    outer_market_share = inner_prob_choice.sum(axis=0)/n_consumer

    market_j = pd.DataFrame({
        'market_id': market,
        'product_id': list(range(n_product)),
        'x': x, 
        'instrument': instrument, 
        'price': price,
        'share': outer_market_share, 
        'share_not_buy': 1 - sum(outer_market_share)
    })
    # print(market)
    all_markets.append(market_j)

print(all_markets)
export_dataset = pd.concat(all_markets, ignore_index=True)
export_dataset.to_csv('simulated_market_data.csv', index=False)

[   market_id  product_id         x  instrument     price     share  \
0          0           0  0.818417    5.377441  4.799286  0.001623   
1          0           1  0.900153    6.203964  5.506785  0.000803   
2          0           2  0.912298    2.269008  2.413187  0.700730   
3          0           3  0.274230    7.827102  6.357809  0.000003   
4          0           4  0.111971    3.608692  2.921372  0.000751   

   share_not_buy  
0        0.29609  
1        0.29609  
2        0.29609  
3        0.29609  
4        0.29609  ,    market_id  product_id         x  instrument     price     share  \
0          1           0  0.835495    9.635902 -4.291078  0.887087   
1          1           1  0.154822    5.866626 -2.733372  0.000137   
2          1           2  0.780627    4.074867 -1.668707  0.000951   
3          1           3  0.120113    7.754223 -3.641703  0.001031   
4          1           4  0.568667    9.259870 -4.203606  0.110794   

   share_not_buy  
0   2.194976e-07  
1   